# ERA5 hourly data on pressure levels

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Select time range

In [ ]:
start = "2022-01"
stop = None  # None: present

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"
request = {
    "format": "netcdf",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": [
        "1",
        "5",
        "20",
        "70",
        "150",
        "225",
        "350",
        "500",
        "650",
        "775",
        "850",
        "925",
        "1000",
    ],
    "time": "00:00",
}
requests = download.update_request_date(request, start=start, stop=stop)

## Download data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_mean,
    chunks={"year": 1},
)

## Compute anomalies

In [ ]:
with xr.set_options(keep_attrs=True):
    ds_anoma = ds - ds.mean("time")
ds_anoma["t"].attrs["long_name"] = ds["t"].attrs["long_name"] + " Anomaly"

## Plot

In [ ]:
lines = ds_anoma["t"].plot(hue="level")
level = ds_anoma["level"]
plt.legend(
    lines,
    level.values,
    bbox_to_anchor=(1, 1),
    loc="upper left",
    title=f"{level.attrs['long_name']} [{level.attrs['units']}]",
)